### Part 5: Create your own strategy *[15 points]*

Imagine today is **31 December 2020** (you can use any data available before 2021). You are a group of fund managers managing a quantitative fund with an **investment universe** consists of **index constituents from Part 1**. Your group wants to **design a new investment strategy and implement it starting from 2021**. The work plan is as follows:
1. Form **benchmark portfolios** based on **Indexation (VW, EW, PW)** and **Modern Portfolio Theory (GMVP and MSRP)** *[5 points]*.
2. Create and backtest a **proprietary trading strategy** that aims to **outperform the market index** *[5 points]*.
3. Keep track of the **out-of-sample performance** of **your group's strategy** as well as **benchmark portfolios** and carry out **performance evaluations** using data available in 2021 so far *[5 points]*.

Your team will **build the prototype of the strategy with Python** and the **source code** below will be inspected by the **model validation quant, Douglas Chung**. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
page = pd.read_html('https://en.wikipedia.org/wiki/DAX')
constit = page[3]['Ticker symbol']
constit.head(3)

0    ADS.DE
1    ALV.DE
2    BAS.DE
Name: Ticker symbol, dtype: object

In [ ]:
back_test_df = pd.DataFrame()